In [1]:
import pandas as pd

In [3]:
def clean_nbs_data_v11(data_source, output_filepath='cleaned_nbs_data.csv'):
    try:
        # Load data
        nbs_data = pd.read_csv(data_source)
        print(f"Structure of raw data: {nbs_data.shape}")

        # Log available columns for debugging
        print(f"Available columns in raw data:\n{list(nbs_data.columns)}")

        # Drop irrelevant columns
        columns_to_drop = ['_tags', '_notes', '_duration', '_id', '_uuid', 'meta/instanceID', 
                           '_submission_time', '_date_modified', '_version', '_submitted_by', 
                           '_total_media', '_media_count', '_media_all_received', '_xform_id']
        nbs_data.drop(columns=columns_to_drop, errors='ignore', inplace=True)

        # Rename columns for consistency
        nbs_data.rename(columns={
            'today': 'Date',
            'STATELABEL': 'State',
            'lgalabel': 'LGA',
            'g_consent/Section_A/market_type': 'Outlet Type',
            'g_consent/Section_A/_gps_latitude': 'Latitude',
            'g_consent/Section_A/_gps_longitude': 'Longitude',
            'sector': 'Sector',
            'VC_ID': 'CONTRIBUTOR ID'  # Contributor ID from VC_ID
        }, inplace=True)

        nbs_data['Country'] = 'Nigeria'  # Default to "Nigeria" for all rows

        # Add a placeholder for Locality if it doesn't exist
        if 'Locality' not in nbs_data.columns:
            nbs_data['Locality'] = None  # Create an empty column or derive it if possible

        # Define corrected food items and associated columns for UOM, Quantity, Price, and Weight
        food_mapping = {
            'g_consent/Section_B1/maize_yellow': {
                'uom': 'g_consent/Section_B1/uom_Ymaize',
                'quantity': 'g_consent/Section_B1/Q_Ymaize',
                'price': 'g_consent/Section_B1/price_Ymaize'
            },
            'g_consent/Section_B2/maize_white': {
                'uom': 'g_consent/Section_B2/uom_Wmaize',
                'quantity': 'g_consent/Section_B2/Q_Wmaize',
                'price': 'g_consent/Section_B2/price_Wmaize'
            },
            'g_consent/Section_B3/sorghum': {
                'uom': 'g_consent/Section_B3/uom_sorghum',
                'quantity': 'g_consent/Section_B3/Q_sorghum',
                'price': 'g_consent/Section_B3/price_sorghum'
            },
            'g_consent/Section_B4/imported_rice': {
                'uom': 'g_consent/Section_B4/uom_imported_rice',
                'quantity': 'g_consent/Section_B4/Q_rice',
                'price': 'g_consent/Section_B4/price_imported_rice'
            },
            'g_consent/Section_B5/local_rice': {
                'uom': 'g_consent/Section_B5/uom_local_rice',
                'quantity': 'g_consent/Section_B5/Q_local_rice',
                'price': 'g_consent/Section_B5/price_local_rice'
            },
            'g_consent/Section_B6/brown_beans': {
                'uom': 'g_consent/Section_B6/uom_brownbeans',
                'quantity': 'g_consent/Section_B6/Q_brownbeans',
                'price': 'g_consent/Section_B6/price_brown_beans'
            },
            'g_consent/Section_B7/White_beans': {
                'uom': 'g_consent/Section_B7/uom_whitebeans',
                'quantity': 'g_consent/Section_B7/Q_whitebeans',
                'price': 'g_consent/Section_B7/price_White_beans'
            },
            'g_consent/Section_B8/garri_confirm': {
                'uom': 'g_consent/Section_B8/uom_garri',
                'quantity': 'g_consent/Section_B8/Q_garri',
                'price': 'g_consent/Section_B8/price_garri'
            },
            'g_consent/Section_B9/yam_confirm': {
                'uom': 'g_consent/Section_B9/uom_yam',
                'quantity': 'g_consent/Section_B9/Q_yam',
                'price': 'g_consent/Section_B9/price_yam'
            },
            'g_consent/Section_B10/Soyabeans': {
                'uom': 'g_consent/Section_B10/uom_soyabeans',
                'quantity': 'g_consent/Section_B10/Q_soyabeans',
                'price': 'g_consent/Section_B10/price_soyabeans'
            }
        }
        
        
        # Prepare long-format data for each food item
        long_format_data = []
        for food_col, mapping in food_mapping.items():
            uom_col = mapping['uom']
            quantity_col = mapping['quantity']
            price_col = mapping['price']

            # Check if required columns exist
            required_columns = [uom_col, quantity_col, price_col]
            missing_columns = [col for col in required_columns if col not in nbs_data.columns]

            if missing_columns:
                print(f"Skipping {food_col}: Missing columns {missing_columns}")
                continue

            temp_df = nbs_data[['Date', 'State', 'CONTRIBUTOR ID', 'LGA', 'Locality', 'Outlet Type', 'Latitude', 'Longitude', 'Country', 'Sector']].copy()
            temp_df['Food Item'] = food_col.split('/')[-1].replace('_', ' ').capitalize()
            temp_df['UOM'] = nbs_data[uom_col]

            # Convert Quantity, Price to numeric
            temp_df['Quantity'] = pd.to_numeric(nbs_data[quantity_col], errors='coerce')
            temp_df['Price'] = pd.to_numeric(nbs_data[price_col], errors='coerce')

            # Extract Weight from UOM (numeric part only) to create UOM2
            temp_df['UOM'] = temp_df['UOM'].astype(str)  # Ensure all values are strings
            temp_df['UOM2'] = temp_df['UOM'].str.extract(r'(\d+\.?\d*)').astype(float)  # Extract numeric part

            # Use UOM2 as Weight
            temp_df['Weight'] = temp_df['UOM2']

            # Calculate Unit Price (Price/Weight) and round to 2 decimal places
            temp_df['UPRICE'] = (temp_df['Price'] / temp_df['Weight']).round(2)
            
            # Add placeholder for price category
            temp_df['Price Category'] = nbs_data['g_consent/Section_A/price_category']  # Assuming this column exists

            # Clean Outlet Type by replacing underscores with spaces
            temp_df['Outlet Type'] = temp_df['Outlet Type'].str.replace('_', ' ', regex=False)
            
            long_format_data.append(temp_df)

        # Concatenate the long-format data
        if long_format_data:
            cleaned_data = pd.concat(long_format_data, ignore_index=True)
        else:
            print("No data was added to long_format_data; check column mappings and data structure.")
            return None

        # Reorder columns as specified
        column_order = [
            'Date', 'State', 'CONTRIBUTOR ID', 'LGA', 'Locality', 'Outlet Type', 'Latitude', 'Longitude',
            'Country', 'Sector', 'Food Item', 'UOM', 'Quantity', 'UOM2', 'Price Category', 'Price', 'Weight', 'UPRICE'
        ]
        cleaned_data = cleaned_data[column_order]

        # Convert 'Date' to datetime format
        cleaned_data['Date'] = pd.to_datetime(cleaned_data['Date'], errors='coerce')

        # Display missing values percentage for all columns
        print("Missing Percentage in Each Feature Before Treatment:")
        missing_percentage = cleaned_data.isna().mean() * 100
        print(missing_percentage.round(2))

        # Apply missing value treatments
        essential_cols = ['State', 'LGA', 'Date', 'Food Item', 'UPRICE', 'UOM', 'UOM2', 'Quantity', 'Price', 'Weight']
        cleaned_data.dropna(subset=essential_cols, inplace=True)

        # Check final structure of cleaned data
        print(f"Structure of cleaned data after treatment: {cleaned_data.shape}")

        # Save cleaned data to CSV
        cleaned_data.to_csv(output_filepath, index=False)
        print(f"\nCleaned data saved to {output_filepath}")

        return cleaned_data
    
    except FileNotFoundError:
        print(f"Error: The file '{data_source}' was not found.")
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [4]:
data_source = r"D:\gigs\nbs\NBS_Crowdsource_Food_Price_Form.csv"

In [5]:
clean_nbs_data_v11(data_source,output_filepath='cleaned_nbs_data_11_27_24.csv')

Structure of raw data: (855, 76)
Available columns in raw data:
['start', 'today', 'deviceid', 'timeStart', 'ZONECODE1', 'STATECODE1', 'ZONELABEL1', 'STATELABEL', 'lga', 'lgalabel', 'sector', 'VC_ID', 'consent', 'g_consent/Section_A/gps', 'g_consent/Section_A/_gps_latitude', 'g_consent/Section_A/_gps_longitude', 'g_consent/Section_A/_gps_altitude', 'g_consent/Section_A/_gps_precision', 'g_consent/Section_A/market_type', 'g_consent/Section_A/price_category', 'g_consent/Section_A/market_time', 'g_consent/Section_B1/maize_yellow', 'g_consent/Section_B1/uom_Ymaize', 'g_consent/Section_B1/Q_Ymaize', 'g_consent/Section_B1/price_Ymaize', 'g_consent/Section_B2/maize_white', 'g_consent/Section_B2/uom_Wmaize', 'g_consent/Section_B2/Q_Wmaize', 'g_consent/Section_B2/price_Wmaize', 'g_consent/Section_B3/sorghum', 'g_consent/Section_B3/uom_sorghum', 'g_consent/Section_B3/Q_sorghum', 'g_consent/Section_B3/price_sorghum', 'g_consent/Section_B4/imported_rice', 'g_consent/Section_B4/uom_imported_rice', 

C:\Users\i\AppData\Local\Temp\ipykernel_15064\247678016.py:145: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cleaned_data['Date'] = pd.to_datetime(cleaned_data['Date'], errors='coerce')


,Date,State,CONTRIBUTOR ID,LGA,Locality,Outlet Type,Latitude,Longitude,Country,Sector,Food Item,UOM,Quantity,UOM2,Price Category,Price,Weight,UPRICE
0,2024-11-27,ONDO,14,AKURE SOUTH,None,Neighborhood shops kiosk,7.231019,5.189926,Nigeria,Urban,Maize yellow,1.4,1.0,1.40,Retail,1700.0,1.40,1214.29
1,2024-11-27,ONDO,14,AKURE SOUTH,None,RoadSide shops street vendors,7.230080,5.187357,Nigeria,Urban,Maize yellow,1.4,1.0,1.40,Retail,1700.0,1.40,1214.29
2,2024-11-27,ONDO,14,AKURE SOUTH,None,RoadSide shops street vendors,7.229785,5.187660,Nigeria,Urban,Maize yellow,1.4,1.0,1.40,Retail,1.0,1.40,0.71
3,2024-11-27,KATSINA,6,KANKIA,None,Neighborhood shops kiosk,12.555025,7.840063,Nigeria,Urban,Maize yellow,1.3,1.0,1.30,Retail,1700.0,1.30,1307.69
4,2024-11-27,KATSINA,6,KANKIA,None,Neighborhood shops kiosk,12.554779,7.839492,Nigeria,Urban,Maize yellow,1.3,1.0,1.30,Retail,1700.0,1.30,1307.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8545,2024-11-27,NASARAWA,15,LAFIA,None,Neighborhood shops kiosk,8.498248,8.529127,Nigeria,Urban,Soyabeans,1.4,3.0,1.40,Retail,2000.0,1.40,1428.57
8546,2024-11-27,NASARAWA,15,LAFIA,None,Specialized stores,8.498058,8.529033,Nigeria,Urban,Soyabeans,1.4,2.0,1.40,Retail,2500.0,1.40,1785.71
8547,2024-11-27,LAGOS,11,LAGOS MAINLAND,None,Neighborhood shops kiosk,6.505522,3.388303,Nigeria,Urban,Soyabeans,0.65,1.0,0.65,Retail,1200.0,0.65,1846.15
8548,2024-11-27,LAGOS,11,LAGOS MAINLAND,None,Neighborhood shops kiosk,6.500628,3.389691,Nigeria,Urban,Soyabeans,0.65,1.0,0.65,Retail,1100.0,0.65,1692.31


In [6]:
pip install nbsfoodpricecleaner==0.0.4

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from nbsfoodpricecleaner import data_cleaner as dc

In [21]:
input_filepath=r"D:\gigs\nbs\NBS_Crowdsource_Food_Price_Form_27th.csv"  # Replace with the actual raw data CSV path
output_filepath="cleaned_data_test.csv"

In [22]:
cleaner = dc.NBSFoodPriceCleaner(input_filepath, output_filepath)

In [23]:
cleaner.load_data()

In [18]:
cleaner.clean_data()

In [19]:
cleaner.save_cleaned_data()

Cleaned data saved to cleaned_data_test.csv


In [25]:
pd.set_option('display.max_columns', 100) 

In [26]:
cleaner.data.head()

,start,today,deviceid,timeStart,ZONECODE1,STATECODE1,ZONELABEL1,STATELABEL,lga,lgalabel,sector,VC_ID,consent,g_consent/Section_A/market_type,g_consent/Section_A/price_category,g_consent/Section_A/market_time,g_consent/Section_B1/maize_yellow,g_consent/Section_B1/uom_Ymaize,g_consent/Section_B1/Q_Ymaize,g_consent/Section_B1/price_Ymaize,g_consent/Section_B2/maize_white,g_consent/Section_B2/uom_Wmaize,g_consent/Section_B2/Q_Wmaize,g_consent/Section_B2/price_Wmaize,g_consent/Section_B3/sorghum,g_consent/Section_B3/uom_sorghum,g_consent/Section_B3/Q_sorghum,g_consent/Section_B3/price_sorghum,g_consent/Section_B4/imported_rice,g_consent/Section_B4/uom_imported_rice,g_consent/Section_B4/Q_rice,g_consent/Section_B4/price_imported_rice,g_consent/Section_B5/local_rice,g_consent/Section_B5/uom_local_rice,g_consent/Section_B5/Q_local_rice,g_consent/Section_B5/price_local_rice,g_consent/Section_B6/brown_beans,g_consent/Section_B6/uom_brownbeans,g_consent/Section_B6/Q_brownbeans,g_consent/Section_B6/price_brown_beans,g_consent/Section_B7/White_beans,g_consent/Section_B7/uom_whitebeans,g_consent/Section_B7/Q_whitebeans,g_consent/Section_B7/price_White_beans,g_consent/Section_B8/garri_confirm,g_consent/Section_B8/uom_garri,g_consent/Section_B8/Q_garri,g_consent/Section_B8/price_garri,g_consent/Section_B9/yam_confirm,g_consent/Section_B9/uom_yam,g_consent/Section_B9/Q_yam,g_consent/Section_B9/price_yam,g_consent/Section_B10/Soyabeans,g_consent/Section_B10/uom_soyabeans,g_consent/Section_B10/Q_soyabeans,g_consent/Section_B10/price_soyabeans,g_consent/seller_phone,gps,_gps_latitude,_gps_longitude,_gps_altitude,_gps_precision,meta/instanceID,_id,_uuid,_submission_time,_date_modified,_tags,_notes,_version,_duration,_submitted_by,_total_media,_media_count,_media_all_received,_xform_id
0,2024-11-27T06:54:30.580+01:00,2024-11-27,collect:8LMG151IuG4hfzbH,2024-11-27T06:54:30.596+01:00,6,28,South West,ONDO,2806,AKURE SOUTH,Urban,14,Yes,Neighborhood_shops_kiosk,Retail,20,Yes,1.4,1.0,1700.0,Yes,1.4,1.0,1600.0,Yes,1.6,1.0,1900.0,Yes,1.5,1.0,3500.0,Yes,1.5,1.0,3000.0,Yes,1.6,1.0,4900.0,Yes,1.6,1.0,2300.0,Yes,1.5,1.0,1000.0,Yes,1.0,1.0,2000.0,Yes,1.6,1.0,2100.0,9.045011e+09,NaN,NaN,NaN,NaN,NaN,uuid:4dfb7602-0d1d-4041-9dc7-d31117ee4bf3,164971702,4dfb7602-0d1d-4041-9dc7-d31117ee4bf3,2024-11-27T06:24:13.063854+00:00,2024-11-27T06:24:13.203916+00:00,NaN,NaN,202411262117,NaN,nbssouthwest,0,0,True,830888
1,2024-11-27T07:09:54.514+01:00,2024-11-27,collect:8LMG151IuG4hfzbH,2024-11-27T07:09:54.527+01:00,6,28,South West,ONDO,2806,AKURE SOUTH,Urban,14,Yes,RoadSide_shops_street vendors,Retail,20,Yes,1.4,1.0,1700.0,Yes,1.4,1.0,1550.0,Yes,1.6,1.0,1900.0,Yes,1.5,1.0,3500.0,Yes,1.5,1.0,2900.0,Yes,1.6,1.0,5000.0,Yes,1.6,1.0,2300.0,Yes,1.5,1.0,950.0,Yes,1.0,1.0,2500.0,Yes,1.6,1.0,2200.0,9.017060e+09,NaN,NaN,NaN,NaN,NaN,uuid:4182962a-63e2-404e-a446-582d6f5e6995,164971703,4182962a-63e2-404e-a446-582d6f5e6995,2024-11-27T06:24:13.640608+00:00,2024-11-27T06:24:13.734339+00:00,NaN,NaN,202411262117,NaN,nbssouthwest,0,0,True,830888
2,2024-11-27T07:25:44.646+01:00,2024-11-27,collect:8LMG151IuG4hfzbH,2024-11-27T07:25:44.649+01:00,6,28,South West,ONDO,2806,AKURE SOUTH,Urban,14,Yes,RoadSide_shops_street vendors,Retail,20,Yes,1.4,1.0,1.0,Yes,1.4,1.0,1700.0,Yes,1.6,1.0,2200.0,Yes,1.5,1.0,3600.0,Yes,1.5,1.0,3000.0,Yes,1.6,1.0,5000.0,Yes,1.6,1.0,2400.0,Yes,1.5,1.0,950.0,Yes,1.0,1.0,2000.0,Yes,1.6,1.0,2200.0,7.033071e+09,NaN,NaN,NaN,NaN,NaN,uuid:6af67aca-fba9-4ad0-aade-3d60f6f6221c,164973585,6af67aca-fba9-4ad0-aade-3d60f6f6221c,2024-11-27T06:37:01.465714+00:00,2024-11-27T06:37:01.575837+00:00,NaN,NaN,202411262117,NaN,nbssouthwest,0,0,True,830888
3,2024-11-27T07:22:57.550+01:00,2024-11-27,collect:FZof3y0ocp893RSv,2024-11-27T07:22:57.566+01:00,3,20,North West,KATSINA,2020,KANKIA,Urban,6,Yes,Neighborhood_shops_kiosk,Retail,10,Yes,1.3,1.0,1700.0,Yes,1.3,1.0,1700.0,Yes,1.5,1.0,1300.0,Yes,1.4,1.0,5300.0,Yes,1.4,1.0,4000.0,No,NaN,NaN,NaN,Yes,1.4,1.0,2500.0,Yes,1.4,1.0,2200.0,No,NaN,NaN,NaN,Yes,1.4,

In [ ]:
from nbsfoodpricecleaner import data_cleaner as dc

# File paths
input_filepath = r"D:\gigs\nbs\NBS_Crowdsource_Food_Price_Form_27th.csv"  # Replace with the actual raw data CSV path
output_filepath = "cleaned_data_test.csv"  # Path to save the cleaned data

# Create an instance of the cleaner
cleaner = dc.NBSFoodPriceCleaner(input_filepath, output_filepath)

# Load the data
cleaner.load_data()

# Clean the data
cleaner.clean_data()

# Save the cleaned data
cleaner.save_cleaned_data()
